# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:
# read in the .csv
ct_data = "..\output_data\cities.csv"
city_df = pd.read_csv(ct_data)
del city_df['Unnamed: 0']
city_df[['Latitude' , 'Longitude']] = city_df[['Latitude', 'Longitude']].apply(pd.to_numeric)
city_df.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather
0,Hilo,19.7297,-155.0900,70.11,92,8,4.00,US,2021-09-12
1,Esperance,-33.8667,121.9000,49.84,72,11,12.08,AU,2021-09-12
2,Cidreira,-30.1811,-50.2056,68.09,76,6,14.52,BR,2021-09-12
3,Busselton,-33.6500,115.3333,50.09,81,96,4.23,AU,2021-09-12
4,Bluff,-46.6000,168.3333,47.80,77,87,24.87,NZ,2021-09-12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
import gmaps

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

# Create the heatmaps (used example form anthony)
fig = gmaps.figure(center=[0,0], zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [62]:
# creating a new dataframe for only those cities that are between 70 and 80 degrees, windspeed less than 10
ideal_city = city_df.loc[(city_df['Tempature']>=70) & (city_df['Tempature'] <=80)]
ideal_city = ideal_city.loc[(ideal_city['Windspeed'] <= 10)]
ideal_city.dropna()
ideal_city.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather
0,Hilo,19.7297,-155.0900,70.11,92,8,4.00,US,2021-09-12
6,Rikitea,-23.1203,-134.9692,72.97,74,26,6.13,PF,2021-09-12
9,Bengkulu,-3.8004,102.2655,76.91,89,66,3.06,ID,2021-09-12
10,Magnolia,33.2671,-93.2393,78.48,51,0,4.05,US,2021-09-12
14,Albany,42.6001,-73.9662,73.62,76,93,6.06,US,2021-09-12


In [63]:
# stor into new hotel_df
hotel_df = ideal_city.reset_index(drop=True)
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather,Hotel Name
0,Hilo,19.7297,-155.0900,70.11,92,8,4.00,US,2021-09-12,
1,Rikitea,-23.1203,-134.9692,72.97,74,26,6.13,PF,2021-09-12,
2,Bengkulu,-3.8004,102.2655,76.91,89,66,3.06,ID,2021-09-12,
3,Magnolia,33.2671,-93.2393,78.48,51,0,4.05,US,2021-09-12,
4,Albany,42.6001,-73.9662,73.62,76,93,6.06,US,2021-09-12,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary
params = {
    "type" : "hotel",
    "keyword": "hotel",
    "radius": "5000",    
    "key": g_key
}



In [52]:
# Create a loop through each city and lat/lon in the datafraem
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row["Longitude"]
    city_name = row["City"]

    params["location"] = f"{lat},{lng}"

     # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: Luanda.
Closest hotel in Luanda is EPIC SANA Luanda.
------------
Retrieving Results for Index 1: Escanaba.
Closest hotel in Escanaba is Super 8 by Wyndham Escanaba.
------------
Retrieving Results for Index 2: Kaili.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: Vaini.
Closest hotel in Vaini is Paradise First Hotel.
------------
Retrieving Results for Index 4: Ponta do Sol.
Closest hotel in Ponta do Sol is Hotel Enotel Baía.
------------
Retrieving Results for Index 5: Chauk.
Closest hotel in Chauk is Bo Khin Guest House.
------------
Retrieving Results for Index 6: Miajadas.
Closest hotel in Miajadas is Hotel Triana.
------------
Retrieving Results for Index 7: Kyabé.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: Jamestown.
Closest hotel in Jamestown is Chautauqua Harbor Hotel.
------------
Retrieving Results for Index 9: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
------------
Ret

In [53]:
hotel_df.head()


,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Date of Weather,Hotel Name
0,Luanda,-8.8368,13.2343,72.45,91,66,2.71,09/11/2021,EPIC SANA Luanda
1,Escanaba,45.7453,-87.0646,70.23,62,50,7.63,09/11/2021,Super 8 by Wyndham Escanaba
2,Kaili,-3.4626,120.2147,71.37,90,85,1.54,09/11/2021,
3,Vaini,-21.2000,-175.2000,73.56,94,90,8.05,09/11/2021,Paradise First Hotel
4,Ponta do Sol,32.6667,-17.1000,76.01,86,36,8.34,09/11/2021,Hotel Enotel Baía


EPIC SANA Luanda
R. da Missão, Luanda


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

KeyError: 'Country'

In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [57]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

NameError: name 'hotel_info' is not defined